## Now lets build an actual image recognition model using transfer learning in Keras.

The model that we’ll be using here is the MobileNet.

Mobile net is a model which gives reasonably good imagenet classification accuracy and occupies very less space. (17 MB according to keras docs).

Dependencies Required :

1. Keras (with tensorflow backend)
2. Numpy
3. Matplotlib
4. Pandas

Data Requirement:

* The training data must be stored in a particular format in order to be fed into the network to train. We will be using ImageDataGenerator, available in keras to train our model on the available data. That way the process becomes much simpler in terms of code.

* There must be a main data folder, inside that data folder, there must be a folder for each class of data containing the corresponding images. The names of the folders must be the names of their respective classes.

The building of a model is a 3 step process:

1. Importing the pre-trained model and adding the dense layers.
2. Loading train data into ImageDataGenerators.
3. Training and Evaluating model.



### First load the dependencies

In [15]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

### Defining our model -
Then import the pre-trained MobileNet model.
* The Mobilenet (trained on the imagenet dataset for a thousand classes) will have a last layer consisting of 1000 neurons (one for each class). We want as many neurons in the last layer of the network as the number of classes we wish to identify. So we discard the 1000 neuron layer and add our own last layer for the network.

This can be done by setting (IncludeTop=False) when importing the model

* We import the MobileNet model without its last layer and add a few dense layers so that our model can learn more complex functions. The dense layers must have the relu activation function and the last layer,which contains as many neurons as the number of classes must have the softmax activation.

So suppose you want to train a dog breed classifier to identify 120 different breeds, we need 120 neurons in the final layer. This can be done using the following code.

In [16]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(120,activation='softmax')(x) #final layer with softmax activation

/home/shreyansh/.local/lib/python3.6/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


### Make Model Architecture -
Next we make a model based on the architecture we have provided.

In [17]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

To check the architecture of our model, we simply need to use this line of code given below.

In [18]:
for i,layer in enumerate(model.layers):
    print(i,layer.name)

0 input_2
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

Now that we have our model, as we will be using the pre-trained weights, that our model has been trained on (imagenet dataset), we have to set all the weights to be non-trainable. We will only be training the last Dense layers that we have made previously. The code for doing this is given below.

In [19]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

### Loading the training data into the ImageDataGenerator -

**ImageDataGenerators** are inbuilt in keras and help us to train our model. We just have to specify the path to our training data and it automatically sends the data for training, in batches. It makes the code much simpler.

For that we need our training data in a particular format as mentioned earlier in the blog.

In [20]:

train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory(r"./MNIST Predictions/", # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 0 images belonging to 1 classes.


*    The **directory** must be set to the path where your ‘n’ classes of folders are present.
*   The **target_size** is the size of your input images, every image will be resized to this size.
*    **color_mode**: if the image is either black and white or grayscale set “grayscale” or if the image has three colour channels, set “rgb”.

*    **batch_size**: No. of images to be yielded from the generator per batch.
*   **class_mode**: Set “binary” if you have only two classes to predict, if not set to“categorical”, in case if you’re developing an Autoencoder system, both input and the output would probably be the same image, for this case set to “input”.

*    **shuffle**: Set True if you want to shuffle the order of the image that is being yielded, else set False.
*    **seed**: Random seed for applying random image augmentation and shuffling the order of the image.

### Training model (after compilation) -

In [24]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,verbose=0, steps_per_epoch=step_size_train,epochs=10)

With this, we will have trained a model. The trained model can then be used to predict which class a new unseen image belongs to, by using -
*model.predict(new_image)*